#Transformer 과제

In [ ]:
!pip install transformers datasets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://repo.ai.gato/registry/repository/pypi-proxy/simple
  Using cached http://repo.ai.gato/registry/repository/pypi-proxy/packages/fsspec/2024.9.0/fsspec-2024.9.0-py3-none-any.whl (179 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.12.0 requires numba-cuda<0.0.18,>=0.0.13, which is not installed.
s3fs 2025.2.0 requires fsspec==2025.2.0.*, but you have fsspec 2024.9.0 which is incompatible.
cudf-cu12 24.12.0 requires pyarrow<19.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 19.0.0 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


##1. 라이브러리 임포트 및 데이터셋 로드

- 질의 응답에 특화된 사전학습 모델을 불러오기 위해 AutoModelForQuestionAnswering 임포트

- 조기 종료를 위한 EarlyStoppingCallback 임포트

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import torch
import numpy as np

# KorQuAD 데이터셋 로드
dataset = load_dataset("KorQuAD/squad_kor_v1")

##2. 데이터셋 전처리

- "xlm-roberta-base" 모델의 토크나이저 불러옴 (XLM-RoBERTa는 다국어로 학습된 모델로 한국어도 비교적 잘 처리함)

In [ ]:
# 모델 체크포인트 지정 및 토크나이저 로드
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]

    tokenized_examples = tokenizer(
        questions,
        contexts,
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        answer = examples["answers"][sample_index]

        if len(answer["answer_start"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])

            sequence_ids = tokenized_examples.sequence_ids(i)
            context_start = sequence_ids.index(1)
            context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

            token_start_index = context_start
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            token_start_index -= 1

            token_end_index = context_end
            while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            token_end_index += 1

            start_positions.append(token_start_index)
            end_positions.append(token_end_index)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

##3. 모델 로드 및 평가 지표 정의

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    start_logits, end_logits = predictions
    start_preds = np.argmax(start_logits, axis=-1)
    end_preds = np.argmax(end_logits, axis=-1)

    if isinstance(labels, tuple):
        start_labels, end_labels = labels
    else:
        start_labels = labels[:, 0]
        end_labels = labels[:, 1]

    exact_matches = (start_preds == start_labels) & (end_preds == end_labels)
    accuracy = np.mean(exact_matches.astype(np.float32))
    return {"accuracy": accuracy}

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##4. TrainingArguments 및 Trainer 생성

- Gradient Accumulation

  - GPU 메모리가 제한적일 때, 한 번의 실제 파라미터 업데이트(optimizer.step()) 전에 여러 배치를 순차적으로 Forward/Backward하여 기울기를 누적하는 방식.
  - 예: gradient_accumulation_steps=2라면, batch size 8의 데이터를 2회 forward/backward 후 최종적으로 batch size 16에 해당하는 gradient를 한꺼번에 업데이트함.
  - 이는 메모리를 효율적으로 사용하면서 효과적으로 대형 배치 사이즈로 학습한 것과 유사한 효과를 낼 수 있습니다.

- Mixed Precision Training(fp16=True)

  - 32-bit 부동소수점 대신 16-bit 부동소수점을 활용해 연산을 가속하고, 메모리 사용량도 절감.
  - NVIDIA GPU(예: Volta, Turing, Ampere 아키텍처)에서 자동 혼합 정밀도(Amp)를 사용하면 손쉽게 적용 가능.
  - 성능(학습 속도) 개선과 함께 GPU 메모리 사용 효율이 높아집니다.

- Warm-up

  - 학습 초반에 학습률을 너무 높게 설정하면 모델이 불안정하게 수렴할 수 있음.
  - warmup_steps=500을 통해 처음 500 스텝 동안은 학습률을 점차 높여 안정적인 학습이 가능하게 함.
- EarlyStoppingCallback

  - 검증 손실(eval_loss)이 개선되지 않으면 미리 학습을 멈춰서 오버피팅을 방지.
  - early_stopping_patience=2는 개선이 없으면 2번 시점 이후 정지하겠다는 의미.

- Best Model Load

  - load_best_model_at_end=True 옵션으로 학습이 끝나면 eval_loss가 가장 좋은(낮은) 체크포인트로 로드.
  - 가장 성능이 좋은 모델 파라미터를 최종 결과로 사용할 수 있음.

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    gradient_accumulation_steps=2,  # Gradient Accumulation 적용
    fp16=True,                     # Mixed Precision Training 적용
    warmup_steps=500,              # Warm-up 단계 추가
    load_best_model_at_end=True,   # 최적 모델 저장 활성화
    metric_for_best_model="eval_loss",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

/usr/gatoai/python/venv/3.10/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_6129/929676005.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


##5. 학습 진행 및 평가

In [ ]:
# 학습 시작
trainer.train()

# 평가 진행 (loss 및 accuracy 출력)
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


Step,Training Loss,Validation Loss,Accuracy
500,1.304500,1.200222,0.633657
1000,0.929700,0.854586,0.713837
1500,0.779900,0.707701,0.745390
2000,0.717100,0.731240,0.761781
2500,0.625700,0.620131,0.788007
3000,0.624700,0.600621,0.792651
3500,0.658400,0.563525,0.796886
4000,0.571500,0.555889,0.798115
4500,0.598200,0.565315,0.795929
5000,0.446600,0.566446,0.808223


Evaluation results: {'eval_loss': 0.5558894872665405, 'eval_accuracy': 0.7981150150299072, 'eval_runtime': 23.891, 'eval_samples_per_second': 306.434, 'eval_steps_per_second': 38.341, 'epoch': 1.0960105217010083}


##6. 테스트

In [ ]:
def predict(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, max_length=384)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()
    answer_ids = inputs["input_ids"][0][start_index : end_index + 1]
    answer = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return answer

test_question = "코로나19의 첫 번째 확진자는 언제 나왔나요?"
test_context = "대한민국의 첫 번째 코로나19 확진자는 2020년 1월 20일에 확인된 환자입니다."
print("Predicted Answer:", predict(test_question, test_context))

Predicted Answer: 2020년 1월 20일에


##어려웠던 점
- 데이터셋이 스팬 추출 기반의 질의응답이었는데 원본 텍스트 기준 문자 위치를 토큰 기준 위치로 매핑해야했다. 이 offset_mapping 로직이 복잡하여서 실수가 많아 구현하는데 오래걸렸다.

- 학습에 몇 시간이 넘게 소요되는 모델과 관련한 과제를 처음 진행해봐서 중간에 런타임이 끊기는 등 처음엔 제대로 시작조차 하지 못하였다. 크지 않는 모델을 사용하고 Early Stopping을 적용한 이후에는 그래도 학습이 비교적 빠르게 끝났다.

##분석 및 고찰
- 한국어에 특화된 모델인 KoBERT같은 모델을 나중 가서 알게 되어서 이걸로 구현을 해봤으면 더 좋지 않았을까 생각한다.

- GPU 자원이 한정되어있어 CS 11강에서 배웠던 하이퍼 파라미터 튜닝 기법을 적용시켜 보았는데 꽤 괜찮은 결과가 나온 것 같다.